<a href="https://colab.research.google.com/github/jjean95/TalentLabs/blob/main/SQL/Titanic/sql_titanic_in_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Loading the sql extention into our environment
%load_ext sql

In [10]:
#import pandas & sqlite3 to read the sql file
import pandas as pd
import sqlite3

In [54]:
#read the sql file using pandas read sql
con = sqlite3.connect('/content/drive/MyDrive/Talent Labs/Foundation/SQL/Titanic/titanic.db')
passengers = pd.read_sql('SELECT * FROM passengers',con)
print(passengers)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male    22      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female    38      1   
2                               Heikkinen, Miss. Laina  female    26      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female    35      1   
4                             Allen, Mr. William Henry    male    35      0   
..                                                 ...     ...   ... 

In [55]:
#inspect the database using sql command, limit by 10
%%sql 
SELECT * 
FROM passengers 
LIMIT 10

 * sqlite://
Done.


index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,None,S
5,6,0,3,"Moran, Mr. James",male,None,0,0,330877,8.4583,None,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,21.075,None,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,11.1333,None,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,237736,30.0708,None,C


In [31]:
# count how many null
%%sql 
SELECT COUNT(*) 
FROM passengers 
WHERE Age IS null

 * sqlite://
Done.


COUNT(*)
177


In [30]:
# count how many null
%%sql 
SELECT COUNT(*) 
FROM passengers 
WHERE Cabin IS null OR Cabin = 0

 * sqlite://
Done.


COUNT(*)
687


In [29]:
# count how many null
%%sql 
SELECT COUNT(*) 
FROM passengers 
WHERE Embarked IS null

 * sqlite://
Done.


COUNT(*)
2


In [34]:
# Survival rate of Titanic
%%sql 
SELECT Survived, count(*) AS survive_count, 
    round(count(*) * 100.0 /(SELECT count(*) FROM passengers),2) AS percentage 
FROM passengers 
GROUP BY Survived

 * sqlite://
Done.


Survived,survive_count,percentage
0,549,61.62
1,342,38.38


In [35]:
#Survival rate of children/elderlies vs adult
%%sql 
WITH Demographics AS 
    (SELECT *, 
      CASE 
      WHEN Age BETWEEN  0 AND 18  then "Children" 
      WHEN Age > 59  then "Elderlies" 
      WHEN Age > 18 AND Age < 60 then "Adult" 
    Else "NULL" 
    END AS Demo 
    FROM passengers) 
SELECT COUNT(*), Demo 
FROM Demographics 
GROUP BY Demo

 * sqlite://
Done.


COUNT(*),Demo
579,Adult
91,Children
44,Elderlies
177,NULL


In [36]:
#Survival rate of children/elderlies vs adult
%%sql           
WITH Demographics AS 
    (SELECT *,
       CASE
       WHEN Age BETWEEN  0 AND 18  then "Children"
       WHEN Age > 59  then "Elderlies" 
       WHEN Age > 18 AND Age < 60 then "Adult"
       Else "NULL"
       END AS Demo
       FROM passengers)
SELECT count(*), Demo, Survived,
          round(count(*) * 100.0 /(SELECT count(*) FROM 
          (SELECT *
          FROM Demographics
          WHERE Demo = 'Adult')),2) AS percentage
FROM Demographics
GROUP BY Demo, Survived
Having Demo = 'Adult'

 * sqlite://
Done.


count(*),Demo,Survived,percentage
347,Adult,0,59.93
232,Adult,1,40.07


In [37]:
#Survival rate of children/elderlies vs adult
%%sql
WITH Demographics AS (SELECT *,
          CASE
          WHEN Age BETWEEN 0 AND 17 then "Children"
          WHEN Age > 59  then "Elderlies" 
          WHEN Age > 18 AND Age < 60 then "Adult"
          Else "NULL"
          END AS Demo
          FROM passengers)
SELECT count(*), Demo, Survived,
          round(count(*) * 100.0 /(SELECT count(*) FROM 
          (SELECT *
          FROM Demographics
          WHERE Demo = 'Children')),2) AS percentage
FROM Demographics
GROUP BY Demo, Survived
Having Demo = 'Children'

 * sqlite://
Done.


count(*),Demo,Survived,percentage
30,Children,0,46.15
35,Children,1,53.85


In [38]:
%%sql
WITH Demographics AS (SELECT *,
          CASE
          WHEN Age BETWEEN 0 AND 17  then "Children"
          WHEN Age > 59  then "Elderlies" 
          WHEN Age > 18 AND Age < 60 then "Adult"
          Else "NULL"
          END AS Demo
          FROM passengers)
SELECT count(*), Demo, Survived,
          round(count(*) * 100.0 /(SELECT count(*) FROM 
          (SELECT *
          FROM Demographics
          WHERE Demo = 'Elderlies')),2) AS percentage
FROM Demographics
GROUP BY Demo, Survived
Having Demo = 'Elderlies'

 * sqlite://
Done.


count(*),Demo,Survived,percentage
30,Elderlies,0,68.18
14,Elderlies,1,31.82


In [39]:
#Gender of survivors
%%sql
SELECT Sex, count(*)
FROM passengers
GROUP BY Sex

 * sqlite://
Done.


Sex,count(*)
female,314
male,577


In [41]:
#Survivors male
%%sql
SELECT Survived, count(*) AS survive_count, 
    round(count(*) * 100.0 / (SELECT count(*) FROM passengers WHERE Sex = 'male'),2) AS percentage
FROM passengers
WHERE Sex = 'male'
GROUP BY Survived

 * sqlite://
Done.


Survived,survive_count,percentage
0,468,81.11
1,109,18.89


In [42]:
#Survivors female
%%sql
SELECT Survived, count(*) AS survive_count, 
    round(count(*) * 100.0 / (SELECT count(*) FROM passengers WHERE Sex = 'female'),2) AS percentage
FROM passengers
WHERE Sex = 'female'
GROUP BY Survived

 * sqlite://
Done.


Survived,survive_count,percentage
0,81,25.8
1,233,74.2


In [43]:
#Class demographics
#class count
%%sql
SELECT Pclass, count(*)
FROM passengers
GROUP BY Pclass

 * sqlite://
Done.


Pclass,count(*)
1,216
2,184
3,491


In [44]:
#1st class
%%sql
SELECT Survived, count(*) AS survive_count,
          round(count(*) * 100.0 / (SELECT count(*) FROM passengers WHERE Pclass = '1'),2) AS percentage
FROM passengers
WHERE Pclass = '1'
GROUP BY Survived

 * sqlite://
Done.


Survived,survive_count,percentage
0,80,37.04
1,136,62.96


In [45]:
#2nd class
%%sql
SELECT Survived, count(*) AS survive_count,
          round(count(*) * 100.0 / (SELECT count(*) FROM passengers WHERE Pclass = '2'),2) AS percentage
FROM passengers
WHERE Pclass = '2'
GROUP BY Survived

 * sqlite://
Done.


Survived,survive_count,percentage
0,97,52.72
1,87,47.28


In [46]:
#3rd class
%%sql
SELECT Survived, count(*) AS survive_count,
          round(count(*) * 100.0 / (SELECT count(*) FROM passengers WHERE Pclass = '3'),2) AS percentage
FROM passengers
WHERE Pclass = '3'
GROUP BY Survived

 * sqlite://
Done.


Survived,survive_count,percentage
0,372,75.76
1,119,24.24


In [47]:
#Embarkation
#Embarkation count
%%sql
SELECT Embarked, count(*)
FROM passengers
GROUP BY Embarked

 * sqlite://
Done.


Embarked,count(*)
None,2
C,168
Q,77
S,644


In [48]:
#C
%%sql
SELECT Survived, count(*) AS survive_count,
          round(count(*) * 100.0 / (SELECT count(*) FROM passengers WHERE Embarked = 'C'),2) AS percentage
FROM passengers
WHERE Embarked = 'C'
GROUP BY Survived

 * sqlite://
Done.


Survived,survive_count,percentage
0,75,44.64
1,93,55.36


In [49]:
#Q
%%sql
SELECT Survived, count(*) AS survive_count,
          round(count(*) * 100.0 / (SELECT count(*) FROM passengers WHERE Embarked = 'Q'),2) AS percentage
FROM passengers
WHERE Embarked = 'Q'
GROUP BY Survived

 * sqlite://
Done.


Survived,survive_count,percentage
0,47,61.04
1,30,38.96


In [50]:
#S
%%sql
SELECT Survived, count(*) AS survive_count,
          round(count(*) * 100.0 / (SELECT count(*) FROM passengers WHERE Embarked = 'S'),2) AS percentage
FROM passengers
WHERE Embarked = 'S'
GROUP BY Survived

 * sqlite://
Done.


Survived,survive_count,percentage
0,427,66.3
1,217,33.7


In [51]:
#Without family
#Parch SibSp count
%%sql
SELECT Parch, SibSp, count(*)
FROM passengers
GROUP BY Parch, SibSp

 * sqlite://
Done.


Parch,SibSp,count(*)
0,0,537
0,1,123
0,2,16
0,3,2
1,0,38
1,1,57
1,2,7
1,3,7
1,4,9
2,0,29


In [52]:
#without family onboard
%%sql
SELECT Survived, count(*) AS survive_count,
          round(count(*) * 100.0 / (SELECT count(*) FROM passengers WHERE Parch = 0 AND SibSp = 0),2) AS percentage
FROM passengers
WHERE Parch = 0 AND SibSp = 0
GROUP BY Survived

 * sqlite://
Done.


Survived,survive_count,percentage
0,374,69.65
1,163,30.35
